In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install pyLDAvis

import re
import numpy as np
import pandas as pd
from pprint import pprint
import datetime
from collections import defaultdict

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk import ngrams

# Gensim
import gensim
from gensim import models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt 

from scipy.stats import pearsonr

import statsmodels
from statsmodels.tsa.stattools import grangercausalitytests

import warnings
warnings.filterwarnings('ignore')

In [ ]:
'''
This cell contains function definitions.
'''

def rel_purity(currPurity, prevPurity):
    return abs(currPurity - prevPurity)/prevPurity


# Select the model and print the topics
def get_topics(lda_model, num_topics=-1, num_words=100, prob_thresh=0.8):
    topics = []
    for topic, topic_words in lda_model.print_topics(num_topics=num_topics, num_words=num_words):
        words = topic_words.split(" + ")
        all_words = []
        all_prob = 0
        for elem in words:
            prob, word = elem.split("*")
            all_prob += float(prob)
            all_words.append(word.split('"')[1])

            if all_prob >= prob_thresh:
                break
        topics.append((topic, all_words))
    
    return topics


'''
Significant portion of grangers_causality_matrix function was taken from stackoverflow post:
https://stackoverflow.com/questions/58005681/is-it-possible-to-run-a-vector-autoregression-analysis-on-a-large-gdp-data-with
'''
def grangers_causality_matrix(data, variables, maxlag=5, test='ssr_ftest', verbose=False):
    dataset = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    lags    = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    
    for c in dataset.columns:
        for r in dataset.index:            
            test_result = grangercausalitytests(data[[r,c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1], 4) for i in range(maxlag)]
            
            if verbose: 
                print(f'Y = {r}, X = {c}, P Values = {p_values}')

            # smaller p-val corresponds to higher f-val
            min_p_value_i = np.argmin(p_values)
            min_p_value = p_values[min_p_value_i]
            dataset.loc[r, c] = min_p_value
            
            lags.loc[r, c] = min_p_value_i
   
    return dataset, lags

def get_causal_vars(data, significance=0.95, getLags=False, getCausalSig=False, verbose=False):
    cols = data.columns[:-1]
    causal_vars = []
    causal_lags = []
    
    for col in cols:
        try:
            gc, lags = grangers_causality_matrix(data[[col, 'LastPrice']], 
                                             variables=[col, 'LastPrice'], 
                                             verbose=False)
        except:
            raise Exception(data[[col, 'LastPrice']])
        
        gc = 1 - gc
        
        col_causes = gc.loc['LastPrice', col] >= significance
        col_causedBy = gc.loc[col, 'LastPrice'] >= significance
        if col_causes or col_causedBy:
            if getCausalSig:
                causal_vars.append((col, max(gc.loc['LastPrice', col], gc.loc[col, 'LastPrice'])))
            else:
                causal_vars.append(col)
            
            if getLags:
                # if sig. granger causality for topic causing ts and ts causing topic, choose whichever is higher
                if col_causes and col_causedBy:
                    if gc.loc['LastPrice', col] >= gc.loc[col, 'LastPrice']:
                        causal_lags.append(lags.loc['LastPrice', col])
                    else:
                        causal_lags.append(lags.loc[col, 'LastPrice'] * -1)
                elif col_causes:
                    causal_lags.append(lags.loc['LastPrice', col])
                else:
                    causal_lags.append(lags.loc[col, 'LastPrice'] * -1)
    if getLags:
        return causal_vars, causal_lags
    
    return causal_vars
                
def get_word_stream(nytimes, topics, causal_topics):
    ct_ws = []
    for ct in causal_topics:
        causal_vocab = list(set(topics[ct][1]))
        date_terms = pd.DataFrame(np.zeros((len(unique_dates), len(causal_vocab))), index=unique_dates, columns=causal_vocab)
        
        for word in causal_vocab:
            date_terms[word] = date_term_cnts[word]
            
        ct_ws.append((ct, date_terms))
    
    return ct_ws

def get_impact_words(topic_wordstream, significance=0.95, verbose=False):
    topic_impact_words = []
    
    first = True
    for topic, ws in topic_wordstream:
        ws_prices = ws.join(stock_prices.set_index('Date')).dropna()        
        ws_gc = get_causal_vars(ws_prices, significance=significance, getCausalSig=True, verbose=verbose)
        
        pos = []
        neg = []
        for word, sig in ws_gc:                
            corr = pearsonr(ws_prices[word], stock_prices['LastPrice'])[0]
            if corr > 0:
                pos.append((word, sig))
            else:
                neg.append((word, sig))
                
        topic_impact_words.append((topic, pos, neg))
    
    return topic_impact_words
        
def construct_prior(impact_words, curr_k, sig=0.95, alter_k=True):
    # find number of topics that we are splitting
    if alter_k:
        new_k = curr_k + len(impact_words)
    else:
        new_k = curr_k
    word_priors = np.zeros((new_k, date_term_cnts.shape[1])) + (1/len(id2word))

    i = 0
    for num, pos, neg in impact_words:
        pos_denom = sum([granger-sig for word, granger in pos])
        neg_denom = sum([granger-sig for word, granger in neg])
        
        if len(pos) < 0.1 * len(neg):
            # num neg words >> num pos
            for word, granger in pos:              
                word_priors[i, id2word.token2id[word]] = 0
            for word, granger in neg:
                word_priors[i, id2word.token2id[word]] = (granger-sig)/neg_denom 
            
        elif len(neg) < 0.1 * len(pos):
            # num pos words >> num neg
            for word, granger in pos:              
                word_priors[i, id2word.token2id[word]] = (granger-sig)/pos_denom 
            for word, granger in neg:
                word_priors[i, id2word.token2id[word]] = 0
            

        for word, granger in pos:              
            word_priors[i, id2word.token2id[word]] = (granger-sig)/pos_denom 
        
        for word, granger in neg:
            word_priors[i + 1, id2word.token2id[word]] = (granger-sig)/neg_denom 
        
        i += 2
    
    # normalize matrix
#     row_sums = word_priors.sum(axis=1, keepdims=True)
#     return word_priors/row_sums
    return word_priors
            
def calculate_purity(pWords, nWords):
    n = float(len(pWords) + len(nWords))
    if n == 0:
        return 0
    pProb = len(pWords)/n
    nProb = len(nWords)/n
    
    pProb = pProb if pProb else 1
    nProb = nProb if nProb else 1
    
    entropy = pProb * np.log2(pProb) + nProb * np.log2(nProb)
    purity = 100 + 100 * entropy
    return purity

def show_plot(x, yData, xaxislabel, yaxislabel, labels, saveAs=None):
    if len(yData) != len(labels):
        raise ValueError("Number of labels should equal number of lines you want to plot")
    
    plt.xlabel(xaxislabel)
    plt.ylabel(yaxislabel)
    plt.grid()
        
    for i in range(len(yData)):
        plt.plot(x, yData[i], label=labels[i])
    plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
    
    if saveAs:
        plt.savefig(saveAs)
    plt.show()

[Timestamp('2000-05-01 05:00:00+0000', tz='UTC'), Timestamp('2000-05-02 05:00:00+0000', tz='UTC'), Timestamp('2000-05-03 05:00:00+0000', tz='UTC'), Timestamp('2000-05-04 05:00:00+0000', tz='UTC'), Timestamp('2000-05-05 05:00:00+0000', tz='UTC'), Timestamp('2000-05-06 05:00:00+0000', tz='UTC'), Timestamp('2000-05-07 05:00:00+0000', tz='UTC'), Timestamp('2000-05-08 05:00:00+0000', tz='UTC'), Timestamp('2000-05-09 05:00:00+0000', tz='UTC'), Timestamp('2000-05-10 05:00:00+0000', tz='UTC'), Timestamp('2000-05-11 05:00:00+0000', tz='UTC'), Timestamp('2000-05-12 05:00:00+0000', tz='UTC'), Timestamp('2000-05-13 05:00:00+0000', tz='UTC'), Timestamp('2000-05-14 05:00:00+0000', tz='UTC'), Timestamp('2000-05-15 05:00:00+0000', tz='UTC'), Timestamp('2000-05-16 05:00:00+0000', tz='UTC'), Timestamp('2000-05-17 05:00:00+0000', tz='UTC'), Timestamp('2000-05-18 05:00:00+0000', tz='UTC'), Timestamp('2000-05-19 05:00:00+0000', tz='UTC'), Timestamp('2000-05-20 05:00:00+0000', tz='UTC'), Timestamp('2000-05-

,date,content
0,2000-05-03 05:00:00+00:00,"[two, years, ago, homer, bush, came, yankee, b..."
1,2000-05-02 05:00:00+00:00,"[texas, record, tell, op, ed, april, paul, bur..."
2,2000-05-01 05:00:00+00:00,"[top, foreign, policy, adviser, gov, george, b..."
3,2000-05-03 05:00:00+00:00,"[aides, gov, george, bush, fought, back, today..."
4,2000-05-03 05:00:00+00:00,"[gov, tommy, thompson, wisconsin, named, chair..."
...,...,...
5801,2000-10-31 05:00:00+00:00,"[new, york, times, cbs, news, poll, var, strin..."
5802,2000-10-31 05:00:00+00:00,"[tick, tock, diner, ted, friedrich, stockbroke..."
5803,2000-11-01 05:00:00+00:00,"[difference, us, vital, issue, would, go, wash..."
5804,2000-11-01 05:00:00+00:00,"[bush, administration, wanted, overturn, would..."


In [ ]:
def ITMTF(threshold=10**-16, mu=1, k=30, alter_k=False, constructTNGraph=False)
    isBaseline = True
    baselineLDAModel = None
    baseline_purity = 0.0
    prevPurity = 100
    purity = 0
    threshold = 10**-16
    mu = 1
    k = 30
    alpha = "auto"
    eta = "auto"
    alter_k = False
    constructTNGraph = False

    all_avg_purities = []
    all_avg_conf = []

    # for k in [10, 20, 30, 40, 10]:
    for mu in [0.51, 0.6, 0.7, 0.8, 0.9, 1]:
        print ("\n\nk: ", k)
        print ("mu: ", mu)

        avg_purities = []
        avg_confidences = []

        num_iter = 0
        while num_iter < 5:
            if isBaseline:
                baselineLDAModel = gensim.models.ldamodel.LdaModel(
                                        corpus=tfidf_corpus,
                                        id2word=id2word,
                                        num_topics=k, 
                                        alpha='auto',  # assuming that topic distribution is assymetric. Not all topics equally represented in corpus.
                                        eta='auto')        

                topics = get_topics(baselineLDAModel, prob_thresh=0.3, num_words=50)
                ct_ws = get_word_stream(nytimes, topics, [i for i in range(k)])

                impact_words = get_impact_words(ct_ws)

                purities = [calculate_purity(topic[1], topic[2]) for topic in impact_words]
                baseline_purity = sum(purities)/len(purities)
                prev_purity = baseline_purity

                print ("Baseline Purity: ", baseline_purity)

                isBaseline = False
    #             avg_purities.append(baseline_purity)
                continue
            else:
                print ("\nNumber topics: ", k)          

                lda_model = gensim.models.ldamodel.LdaModel(
                                        corpus=tfidf_corpus,
                                        id2word=id2word,
                                        num_topics=k, 
                                        alpha=alpha,  # assuming that topic distribution is assymetric. Not all topics equally represented in corpus.
                                        eta=eta,
                                        decay=mu)

            topics = get_topics(lda_model, prob_thresh=0.3, num_words=50)
            document_topics = lda_model.get_document_topics(corpus)
            date_doc_topics = list(zip(nytimes["Date"], lda_model.get_document_topics(corpus)))

            # for any given day, you look at all the diff topics and identify the prob of that topic
            date_topic_prob = np.zeros((len(unique_dates), k))
            for date, article in date_doc_topics:
                i = unique_dates.index(date)
                for topic, prob in article:
                    date_topic_prob[i][topic] += prob 

            date_topic = pd.DataFrame(date_topic_prob, index=unique_dates)
            date_topic["Date"] = unique_dates

            date_topic_prices = date_topic.set_index('Date').join(stock_prices.set_index('Date')).dropna()
            causal_topics, ct_lags = get_causal_vars(date_topic_prices, getLags=True)
            ct_ws = get_word_stream(nytimes, topics, causal_topics)

            impact_words = get_impact_words(ct_ws)

            # Calculate Purity
            purities = [calculate_purity(topic[1], topic[2]) for topic in impact_words]
            if len(purities) == 0:
                # Try again
                continue
                avg_purity= 0
            else:
                avg_purity = sum(purities)/len(purities)

            prevPurity = purity
            purity = avg_purity

            avg_purities.append(purity)
            print ("Purity: ", purity)  

            # Calculate Confidence
            all_conf = 0.0
            num_words = 0
            for topic, pos, neg in impact_words:
                num_words += len(pos) + len(neg)
                for words, gc in pos:
                    all_conf += gc

                for words, gc in neg:
                    all_conf += gc

            avg_confidences.append(all_conf/num_words*100 if num_words else 0)
            print ("Avg. Conf: ", all_conf/num_words*100 if num_words else 0)


            # Prior for next iteration
            eta = construct_prior(impact_words, k, alter_k=alter_k)

            # adjust num topics
            if alter_k:
                k += len(impact_words)

            num_iter += 1

        all_avg_purities.append(avg_purities)
        all_avg_conf.append(avg_confidences)

        # reset params
        isBaseline = True
        eta = "auto"

        if constructTNGraph:
            if k == 40:
                alter_k=True
    
    return lda_model, baselineLDAModel, all_avg_purities, all_avg_conf


[[('ago', 0.07712049873418031),
  ('awesome', 0.23220574510227418),
  ('backup', 0.2198823985449398),
  ('backups', 0.2515408271170864),
  ('bases', 0.19264069440348208),
  ('bellinger', 0.27548950382241366),
  ('bench', 0.1896343958919212),
  ('bush', 0.007894722475376273),
  ('came', 0.08612993720379283),
  ('catcher', 0.26148042600790294),
  ('clay', 0.2135830725972484),
  ('games', 0.1562902360625982),
  ('girardi', 0.27548950382241366),
  ('homer', 0.21658937110880933),
  ('jim', 0.1245222966630543),
  ('joe', 0.1146922085996351),
  ('leyritz', 0.27548950382241366),
  ('speed', 0.17969479700110466),
  ('stole', 0.20587332073042908),
  ('strength', 0.13402729061444735),
  ('turner', 0.2108175460504276),
  ('two', 0.04788545375938528),
  ('versatility', 0.27548950382241366),
  ('whose', 0.0887458288821732),
  ('yankee', 0.20825706839694694),
  ('yankees', 0.19264069440348208),
  ('years', 0.05159983565074285)]]

In [ ]:
# k_labels = ["10", "20", "30", "40", "varying tn"]
# show_plot(range(1, 7), all_avg_purities,    "Iteration", "Average Purity", k_labels, saveAs="purity_varying_tn_2.png")
# show_plot(range(1, 7), all_avg_conf, "Iteration", "Average Confidence", k_labels, saveAs="confidence_varying_tn_1.png")

mu_labels = ["0.51", "0.6", "0.7", "0.8", "0.9", "1"]
show_plot(range(1, 6), np.array(all_avg_purities[5:]), "Iteration", "Average Purity", mu_labels, saveAs="purity_mu.png")
show_plot(range(1, 6), np.array(all_avg_conf[5:]), "Iteration", "Average Confidence", mu_labels, saveAs="confidence_mu.png")

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(baselineLDAModel, corpus, id2word)
pyLDAvis.save_html(vis, 'baselineLDA_mu.html')
vis

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(vis, 'lda_mu.html')
vis